# Feature Extraction

# Using YOLOv9

In [ ]:
import os
import shutil
import subprocess
import re

def run_yolov9_detect(model, base_dir, video):
    """Run the yolov9 detect.py script with specified parameters."""
    command = [
        'python3', 'yolov9-main/detect.py', 
        '--source', f'{os.path.join(base_dir, video)}', 
        '--img', '640', 
        '--weights', f'yolov9-main/yolov9-{model}-converted.pt', 
        '--name', f'{video}_{model}', 
        '--save-txt', 
        '--save-conf', 
        '--save-crop',
        '--nosave'
    ]

    try:
        result = subprocess.run(command, check=True, capture_output=True, text=True)
        print(f"Script output:\n{result.stdout}")
    except subprocess.CalledProcessError as e:
        print(f"Script failed with error:\n{e.stderr}")
        
def merge_results(yolo_output_dir, folders, models):
    merged_dir = 'object_detection_results'
    os.makedirs(merged_dir, exist_ok=True)
    
    base_folders = set()
    # Regular expression to match strings starting with 5 digits
    pattern = re.compile(r'^\d{5}')
    for folder in folders:
        match = pattern.match(folder)
        if match:
            prefix = match.group(0)
            base_folders.add(prefix)
    
    for video in base_folders:
        video_merged_dir = os.path.join(merged_dir, video)
        os.makedirs(video_merged_dir, exist_ok=True)
        
        labels_merged_dir = os.path.join(video_merged_dir, 'labels')
        crops_merged_dir = os.path.join(video_merged_dir, 'crops')
        os.makedirs(labels_merged_dir, exist_ok=True)
        os.makedirs(crops_merged_dir, exist_ok=True)

        all_detections = {}

        for model in models:
            model_result_dir = os.path.join(yolo_output_dir, f'{video}_{model}')
            model_labels_dir = os.path.join(model_result_dir, 'labels')
            model_crops_dir = os.path.join(model_result_dir, 'crops')

            if not os.path.exists(model_labels_dir) or not os.path.exists(model_crops_dir):
                continue

            # Process label files
            for label_file in os.listdir(model_labels_dir):
                if label_file.endswith('.txt'):
                    label_path = os.path.join(model_labels_dir, label_file)
                    with open(label_path, 'r') as f:
                        lines = f.readlines()

                    if label_file not in all_detections:
                        all_detections[label_file] = []
                    
                    for line in lines:
                        parts = line.strip().split()
                        class_id = parts[0]
                        if class_id not in all_detections[label_file]:
                            all_detections[label_file].append((model, line))

        # Merge and copy images
        for label_file, detections in all_detections.items():
            label_merged_path = os.path.join(labels_merged_dir, label_file)
            with open(label_merged_path, 'w') as f_out:
                for idx, (model, line) in enumerate(detections):
                    parts = line.strip().split()
                    class_id = parts[0]

                    src_img_name = label_file.replace('.txt', '') + f"{idx+1}.jpg"
                    src_img_path = os.path.join(base_dir, f'{video}_{model}', 'crops', class_id, src_img_name)
                    
                    new_img_name = f"{os.path.splitext(label_file)[0]}_{model}_{idx+1}.jpg"
                    new_img_path = os.path.join(crops_merged_dir, class_id)
                    os.makedirs(new_img_path, exist_ok=True)
                    dst_img_path = os.path.join(new_img_path, new_img_name)

                    if os.path.exists(src_img_path):
                        shutil.copy(src_img_path, dst_img_path)
                        parts = line.strip().split()
                        f_out.write(' '.join(parts) + '\n')
        
base_dir = 'keyframes_'
models = ['c', 'e']
# Create a list of directories in the keyframes folder
directories = [f for f in os.listdir(base_dir)]
directories.remove('.DS_Store')
print(directories)

# Run detection for each video and model
# for video in directories:
#     for model in models:
#         run_yolov9_detect(model, base_dir, video)

# Merge results
yolo_output_dir = 'yolov9-main/runs/detect'
folders = [f for f in os.listdir(yolo_output_dir)]
folders.remove('.DS_Store')
merge_results(yolo_output_dir, folders, models)

# Color Detection using BLIP

In [ ]:
from transformers import BlipProcessor, BlipForConditionalGeneration
import torch
import cv2
import re

# Load the BLIP model and processor
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

def generate_caption(image_path, prompt="Describe the colors."):
    raw_image = cv2.imread(image_path)
    raw_image = cv2.cvtColor(raw_image, cv2.COLOR_BGR2RGB)
    
    # Prepare image and prompt for BLIP
    inputs = processor(images=raw_image, return_tensors="pt")
    
    # Generate caption
    out = model.generate(**inputs)
    caption = processor.decode(out[0], skip_special_tokens=True)
    return caption

def extract_colors(caption):
    colors = ["red", "green", "blue", "yellow", "white", "black", "orange", "purple", "brown", "gray", "pink"]
    found_colors = [color for color in colors if re.search(r'\b' + color + r'\b', caption)]
    return found_colors


# Load your cropped image
image_path = 'yolov9-main/runs/detect/00120_e/crops/backpack/00120_Scene-77.jpg'  # Replace with the path to your image
target_label = 'backpack'  # Replace with the target label (e.g., 'boat', 'backpack')

# Generate caption for the image
caption = generate_caption(image_path)
print("Generated Caption:", caption)

# Extract colors from the caption
colors_in_caption = extract_colors(caption)
print("Colors found in caption:", colors_in_caption)


# Color detection using three most dominant colors

In [ ]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Predefined list of colors and their RGB values
COLOR_NAMES = {
    "red": [255, 0, 0],
    "green": [0, 255, 0],
    "blue": [0, 0, 255],
    "yellow": [255, 255, 0],
    "white": [255, 255, 255],
    "black": [0, 0, 0],
    "orange": [255, 165, 0],
    "purple": [128, 0, 128],
    "brown": [165, 42, 42],
    "gray": [128, 128, 128],
    "pink": [255, 192, 203]
}

def closest_color_name(rgb_color):
    # Calculate the Euclidean distance between the given color and all predefined colors
    distances = {name: np.linalg.norm(np.array(rgb_color) - np.array(rgb_val)) for name, rgb_val in COLOR_NAMES.items()}
    # Find the color with the smallest distance
    closest_color = min(distances, key=distances.get)
    return closest_color

def get_dominant_colors(image_path, k=3):
    # Read the image
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Reshape the image to be a list of pixels
    pixels = image.reshape(-1, 3)

    # Apply KMeans clustering to find the most dominant colors
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(pixels)

    # Get the cluster centers (the dominant colors)
    colors = kmeans.cluster_centers_

    # Get the number of pixels assigned to each cluster
    counts = np.bincount(kmeans.labels_)

    # Get the percentage of each color
    percentages = counts / sum(counts)

    # Combine colors and percentages
    dominant_colors = sorted(zip(colors, percentages), key=lambda x: -x[1])

    # Map RGB colors to closest color names
    named_dominant_colors = [(closest_color_name(color), percentage) for color, percentage in dominant_colors]

    return named_dominant_colors

def plot_colors(dominant_colors):
    # Create a square showing the dominant colors
    fig, ax = plt.subplots(1, 1, figsize=(12, 2),
                           subplot_kw=dict(xticks=[], yticks=[], frame_on=False))

    bar = np.zeros((50, 300, 3), dtype=np.uint8)
    start = 0

    for color_name, percentage in dominant_colors:
        color_rgb = COLOR_NAMES[color_name]
        end = start + int(percentage * bar.shape[1])
        bar[:, start:end] = color_rgb
        start = end

    ax.imshow(bar)
    plt.show()

# Example usage
image_path = 'yolov9-main/runs/detect/00120_e/crops/person/00120_Scene-273.jpg'  # Replace with the path to your image

# Get the three most dominant colors
dominant_colors = get_dominant_colors(image_path, k=3)

# Print the colors and their dominance
for color_name, percentage in dominant_colors:
    print(f"Color: {color_name}, Percentage: {percentage*100:.2f}%")

# Plot the dominant colors
plot_colors(dominant_colors)


# OCR using EasyOCR

In [ ]:
import cv2
import easyocr
import difflib
import json

def ocr_video(video_path, output_file, similarity_threshold=0.8, frame_step=10):
    # Initialize the video capture and OCR reader
    cap = cv2.VideoCapture(video_path)
    reader = easyocr.Reader(['en'])

    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    # Initialize variables to store the results
    results = []

    prev_text = ""
    current_block = None

    for i in range(0, frame_count, frame_step):
        ret, frame = cap.read()
        if not ret:
            break
        
        # Skip frames until we reach the next frame of interest
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        
        # Calculate the timestamp for the current frame
        timestamp = i / fps

        # Convert the frame to RGB (easyocr works on RGB images)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Perform OCR on the frame
        ocr_result_list = reader.readtext(frame_rgb, detail=0)
        ocr_result = ' '.join(ocr_result_list).strip().lower()

        # Skip frames with no text
        if not ocr_result:
            continue

        # Calculate similarity with the previous text
        similarity = difflib.SequenceMatcher(None, prev_text, ocr_result).ratio()

        if similarity >= similarity_threshold:
            # If the text is similar enough, update the end time of the current block
            if current_block:
                current_block['end_time'] = timestamp + (frame_step / fps)
            else:
                current_block = {
                    'text': ocr_result,
                    'start_time': timestamp,
                    'end_time': timestamp + (frame_step / fps)
                }
        else:
            # If the text is different enough, finalize the current block and start a new one
            if current_block:
                results.append(current_block)
            current_block = {
                'text': ocr_result,
                'start_time': timestamp,
                'end_time': timestamp + (frame_step / fps)
            }
        prev_text = ocr_result

    # Finalize the last block
    if current_block:
        results.append(current_block)

    cap.release()

    # Save results to a JSON file
    with open(output_file, 'w') as f:
        json.dump(results, f, indent=4)

    return results

# Example usage
video_path = 'preprocessed_videos/00102/00102.mp4'
output_file = 'ocr_results.json'
similarity_threshold = 0.5
frame_step = 10
ocr_results = ocr_video(video_path, output_file, similarity_threshold, frame_step)

# for result in ocr_results:
#     print(f"Text: {result['text']}, Start Time: {result['start_time']:.2f}, End Time: {result['end_time']:.2f}")


# Search for a string in OCR results

In [ ]:
import json
import difflib

def compare_string_with_ocr_results(ocr_file, input_string, similarity_threshold=0.8):
    # Load OCR results from the file
    with open(ocr_file, 'r') as f:
        ocr_results = json.load(f)

    # Normalize the input string
    input_string = input_string.strip().lower()

    matching_results = []

    for result in ocr_results:
        ocr_text = result['text']
        similarity = difflib.SequenceMatcher(None, ocr_text, input_string).ratio()

        if similarity >= similarity_threshold:
            matching_results.append({
                'text': ocr_text,
                'start_time': result['start_time'],
                'end_time': result['end_time']
            })

    return matching_results

# Example usage
ocr_file = 'ocr_results.json'
input_string = 'Get reliable diving gear'
similarity_threshold = 0.6

matching_results = compare_string_with_ocr_results(ocr_file, input_string, similarity_threshold)

for result in matching_results:
    print(f"Matching Text: {result['text']}, Start Time: {result['start_time']:.2f}, End Time: {result['end_time']:.2f}")


# Using CLIP

In [ ]:
import os
import torch
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
import torchvision.transforms as T
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from sklearn.metrics import confusion_matrix, recall_score
import matplotlib.pyplot as plt
import seaborn as sns
import xml.etree.ElementTree as ET

# Load models and processors
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
faster_rcnn = fasterrcnn_resnet50_fpn(pretrained=True).to(device)
faster_rcnn.eval()

# Define the folder containing images
frame_dir = '00110'
annotation_dir = '00110'

# Define the queries for evaluation
queries = ["A photo of a person", "A photo of a bird", "A photo of a truck", "A photo of a horse", "A photo of a car"]
query_labels = ["person", "bird", "truck", "horse", "car"]

# Mapping to unify bus and train as truck
class_mapping = {
    "bus": "truck",
    "train": "truck"
}

def get_ground_truth_labels(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    labels = []
    boxes = []
    for member in root.findall('object'):
        label = member.find('name').text
        if label in class_mapping:
            label = class_mapping[label]
        if label in query_labels:
            labels.append(label)
            bndbox = member.find('bndbox')
            xmin = int(bndbox.find('xmin').text)
            ymin = int(bndbox.find('ymin').text)
            xmax = int(bndbox.find('xmax').text)
            ymax = int(bndbox.find('ymax').text)
            boxes.append([xmin, ymin, xmax, ymax])
    return labels, boxes

def iou(box1, box2):
    """Calculate Intersection Over Union (IOU) of two bounding boxes."""
    x1, y1, x2, y2 = box1
    x1_p, y1_p, x2_p, y2_p = box2

    xi1 = max(x1, x1_p)
    yi1 = max(y1, y1_p)
    xi2 = min(x2, x2_p)
    yi2 = min(y2, y2_p)
    inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)

    box1_area = (x2 - x1) * (y2 - y1)
    box2_area = (x2_p - x1_p) * (y2_p - y1_p)
    union_area = box1_area + box2_area - inter_area

    iou = inter_area / union_area
    return iou

true_labels = []
predicted_labels = []

for frame_name in os.listdir(frame_dir):
    if frame_name.endswith(".jpg"):
        frame_path = os.path.join(frame_dir, frame_name)
        annotation_path = os.path.join(annotation_dir, frame_name.replace(".jpg", ".xml"))
        
        img = Image.open(frame_path).convert("RGB")
        if img is None:
            print(f"Error reading image: {frame_path}")
            continue
        
        # Get ground truth labels and boxes
        if not os.path.exists(annotation_path):
            print(f"Annotation file not found: {annotation_path}")
            continue
        gt_labels, gt_boxes = get_ground_truth_labels(annotation_path)

        # Convert image to tensor
        transform = T.Compose([T.ToTensor()])
        img_tensor = transform(img).to(device)

        # Generate bounding boxes using Faster R-CNN
        with torch.no_grad():
            predictions = faster_rcnn([img_tensor])
        pred_boxes = predictions[0]['boxes'].cpu().numpy()
        pred_scores = predictions[0]['scores'].cpu().numpy()

        detected_labels = []
        detected_boxes = []

        for box, score in zip(pred_boxes, pred_scores):
            if score > 0.5:  # Adjust threshold as needed
                xmin, ymin, xmax, ymax = box
                cropped_img = img.crop((xmin, ymin, xmax, ymax))
                inputs = clip_processor(text=queries, images=cropped_img, return_tensors="pt", padding=True).to(device)
                outputs = clip_model(**inputs)
                logits_per_image = outputs.logits_per_image.softmax(dim=1).detach().cpu().numpy().flatten()

                best_idx = logits_per_image.argmax()
                detected_label = query_labels[best_idx]
                confidence = logits_per_image[best_idx]

                detected_labels.append((detected_label, confidence, [xmin, ymin, xmax, ymax]))

        # For visualization and evaluation
        frame_true_labels = []
        frame_predicted_labels = []
        matched_predictions = [False] * len(detected_labels)

        for label, gt_box in zip(gt_labels, gt_boxes):
            frame_true_labels.append(label)
            matched = False
            for i, (detected_label, _, detected_box) in enumerate(detected_labels):
                if iou(gt_box, detected_box) > 0.5 and not matched_predictions[i]:
                    frame_predicted_labels.append(detected_label)
                    matched_predictions[i] = True
                    matched = True
                    break
            if not matched:
                frame_predicted_labels.append("none")

        for i, (detected_label, _, detected_box) in enumerate(detected_labels):
            if not matched_predictions[i]:
                frame_true_labels.append("none")
                frame_predicted_labels.append(detected_label)

        true_labels.extend(frame_true_labels)
        predicted_labels.extend(frame_predicted_labels)

# Print the true and predicted labels for debugging
print("True Labels:", true_labels)
print("Predicted Labels:", predicted_labels)

# Ensure the lengths are equal
min_length = min(len(true_labels), len(predicted_labels))
filtered_true_labels = true_labels[:min_length]
filtered_predicted_labels = predicted_labels[:min_length]

# Calculate confusion matrix
conf_matrix = confusion_matrix(filtered_true_labels, filtered_predicted_labels, labels=query_labels + ["none"])
print("Confusion Matrix:")
print(conf_matrix)

# Calculate recall for each class
recall = recall_score(filtered_true_labels, filtered_predicted_labels, average=None, labels=query_labels)
print("Recall for each class:")
print(recall)

# Weighted average recall
weighted_recall = recall_score(filtered_true_labels, filtered_predicted_labels, average='weighted', labels=query_labels)
print("Weighted Recall:")
print(weighted_recall)

# Plotting the confusion matrix
plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt="d", xticklabels=query_labels + ["none"], yticklabels=query_labels + ["none"], cmap="Blues")
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()


# Using CLIP and BLIP

In [ ]:
import os
import torch
from PIL import Image
from transformers import CLIPProcessor, CLIPModel, BlipProcessor, BlipForConditionalGeneration
import torchvision.transforms as T
from torchvision.models.detection import FasterRCNN, fasterrcnn_resnet50_fpn
import matplotlib.pyplot as plt
from pymongo import MongoClient, errors
import datetime

# Define the folder containing images
folder_path = "keyframes/00102"

# Define the paths to the weights
fasterrcnn_weights_path = "weights/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth"
resnet50_weights_path = "weights/resnet50-0676ba61.pth"

# Check if the files exist
assert os.path.exists(fasterrcnn_weights_path), "Faster R-CNN weights file not found!"
assert os.path.exists(resnet50_weights_path), "ResNet50 weights file not found!"

# Print paths to verify
print(f"Faster R-CNN weights path: {fasterrcnn_weights_path}")
print(f"ResNet50 weights path: {resnet50_weights_path}")

# Load the ResNet50 backbone with local weights
from torchvision.models import resnet50
backbone = resnet50(pretrained=False)
backbone_state_dict = torch.load(resnet50_weights_path, map_location=torch.device('cpu'))

# Remove the fully connected layer weights from the state dictionary
backbone_state_dict.pop("fc.weight", None)
backbone_state_dict.pop("fc.bias", None)

# Load the state dictionary with strict=False to ignore missing keys
backbone.load_state_dict(backbone_state_dict, strict=False)

# Create a custom backbone with FPN from the loaded ResNet50 backbone
from torchvision.models.detection.backbone_utils import resnet_fpn_backbone

# Use the backbone with FPN, ensuring it uses the locally loaded weights
backbone_with_fpn = resnet_fpn_backbone('resnet50', pretrained=False, norm_layer=torch.nn.BatchNorm2d)
backbone_with_fpn.body.load_state_dict(backbone.state_dict(), strict=False)

# Load the Faster R-CNN model with the custom backbone
detection_model = FasterRCNN(backbone=backbone_with_fpn, num_classes=91)  # Use the backbone explicitly
detection_model.load_state_dict(torch.load(fasterrcnn_weights_path, map_location=torch.device('cpu')))
detection_model.eval()

# Load the CLIP model and processor
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Load the BLIP captioning model and processor
blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

# MongoDB setup with error handling
try:
    client = MongoClient('mongodb://localhost:27017/', serverSelectionTimeoutMS=5000)
    client.server_info()  # Trigger exception if cannot connect to db
    db = client['object_detection']
    collection = db['detected_objects']
except errors.ServerSelectionTimeoutError as err:
    print("Failed to connect to MongoDB server:", err)
    exit(1)

# Transform for the object detection model
transform = T.Compose([T.ToTensor()])

# Function to generate captions using BLIP
def generate_caption(image):
    inputs = blip_processor(images=image, return_tensors="pt")
    out = blip_model.generate(**inputs)
    caption = blip_processor.decode(out[0], skip_special_tokens=True)
    return caption

# Process each image in the folder
for filename in os.listdir(folder_path):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        image_path = os.path.join(folder_path, filename)
        image = Image.open(image_path)

        # Transform image for the detection model
        image_tensor = transform(image)

        # Get bounding boxes
        with torch.no_grad():
            detections = detection_model([image_tensor])[0]

        # Filter out low-confidence detections
        threshold = 0.5
        boxes = [box for box, score in zip(detections['boxes'], detections['scores']) if score > threshold]

        detected_objects = []

        # Use BLIP to generate captions for objects within bounding boxes
        for box in boxes:
            xmin, ymin, xmax, ymax = box.int().numpy()
            cropped_image = image.crop((xmin, ymin, xmax, ymax))
            caption = generate_caption(cropped_image)
            inputs = clip_processor(text=[caption], images=cropped_image, return_tensors="pt", padding=True)
            outputs = clip_model(**inputs)
            probs = outputs.logits_per_image.softmax(dim=1).detach().cpu().numpy()[0]
            detected_label = caption  # Use the generated caption as the label
            confidence = probs.max()

            detected_objects.append({
                "box": [xmin, ymin, xmax, ymax],
                "label": detected_label,
                "confidence": float(confidence)
            })

            # Prepare the data to be stored in MongoDB
            detected_object = {
                "filename": filename,
                "label": detected_label,
                "confidence": float(confidence),
                "box": [int(xmin), int(ymin), int(xmax), int(ymax)],
                "timestamp": datetime.datetime.utcnow()
            }

            # Insert the data into MongoDB
            collection.insert_one(detected_object)
            print(f"Image: {filename}, Detected {detected_label} with confidence {confidence:.4f} within box {box}")

        # Optionally, display the image with detected bounding boxes and labels
        plt.imshow(image)
        plt.axis('off')
        ax = plt.gca()
        for obj in detected_objects:
            xmin, ymin, xmax, ymax = obj['box']
            detected_label = obj['label']
            confidence = obj['confidence']
            rect = plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin, fill=False, color='red', linewidth=2)
            ax.add_patch(rect)
            plt.text(xmin, ymin, f'{detected_label} {confidence:.2f}', bbox=dict(facecolor='yellow', alpha=0.5))

        plt.show()
